<img src= "img/pipelines.png" style="height:450px">


[Image Source](https://towardsdatascience.com/using-functiontransformer-and-pipeline-in-sklearn-to-predict-chardonnay-ratings-9b13fdd6c6fd)

__Agenda__

- Pipelines and Composite estimators

- Why do we need them?

- How to use them in sklearn: accessing a particular object in pipe and changing parameters

- ROC Curve

- Area under ROC Curve

In [1]:
import pandas as pd
import numpy as np

# Pipelines

__What is a Pipeline?__

_Transformers:_ Any object with .transform method. Ex: StandardScaler, OneHotEncoder.

_Estimators:_ Any object with predict method. Ex: RandomForestClassifier, LinearRegression etc.

_Pipelines:_ A tool for combining transformers with estimators. 


__Why do we need pipelines?__

- Convenience and encapsulation

Even though we train 10 transformers and 5 estimators we will call fit and predict once.

- Joint parameter selection - here emphasize preprocessing part

We can put pipelines into gridsearch and find best parameters for all the estimators at once.

- Safety

Pipelines help avoid leaking statistics from your test data into trained model.

[Pipelines and Composite Estimators](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)



## Usage of Pipelines

The Pipeline is built using a list of (key, value) pairs, where the key is a string containing the name you want to give this step and value is an estimator object:

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

estimators = [('imputer', SimpleImputer()),
              ('scaler', StandardScaler()),
              ('clf', LogisticRegression())]

pipe = Pipeline(estimators)

pipe

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('clf', LogisticRegression())])

In [3]:
print(pipe[0])
print(pipe['scaler'])
print(pipe.steps)
print(pipe.steps[0])
print(pipe.steps[0][1])
print(pipe.named_steps.imputer)

SimpleImputer()
StandardScaler()
[('imputer', SimpleImputer()), ('scaler', StandardScaler()), ('clf', LogisticRegression())]
('imputer', SimpleImputer())
SimpleImputer()
SimpleImputer()


__Your Turn__

- Create your own pipeline. You can use the same transformers and estimators with different parameters and 'name'.

- You can create a new pipe with a scaler also.

In [4]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

your_estimators = [('knnimputer', KNNImputer()),
                   ('scaler', MinMaxScaler((0,1))),
                   ('clf', DecisionTreeClassifier())]

your_pipe = Pipeline(your_estimators)

Sklearn also gives us "make_pipeline" which is almost the same thing but with make_pipeline you don't have to give names.

__Your Turn__

-  [Check documentation: 6.1.1.1.1 Construction](https://scikit-learn.org/stable/modules/compose.html) and use `make_pipeline` to construct a pipeline.

In [5]:
from sklearn.pipeline import make_pipeline

In [6]:
your_make_pipe = None

## Accessing steps

We have multiple ways to access and object in the pipeline

In [7]:
## note that these will all give the simple imputer object
print(pipe[0])

print(pipe['imputer'])

pipe.steps[0][1]

SimpleImputer()
SimpleImputer()


SimpleImputer()

In [8]:
## We can also access a particular object by named_steps

pipe.named_steps.imputer

SimpleImputer()

In [9]:
## We can 'slice' pipelines to create sub-pipes

pipe[1:]

Pipeline(steps=[('scaler', StandardScaler()), ('clf', LogisticRegression())])

## Access to the parameters

Parameters of the estimators in the pipeline can be accessed using the 
"estimator__parameter" syntax.

In [10]:
pipe.named_steps.clf.C

1.0

In [11]:
pipe.set_params(clf__C = 10)

pipe

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('clf', LogisticRegression(C=10))])

In [12]:
pipe.named_steps.clf.C

10

## Pipelines in action

In [13]:
from sklearn.model_selection import train_test_split

[Dataset info](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)

In [14]:
df = pd.read_csv('./data/heart.csv').drop(columns='Unnamed: 0')

In [15]:
print(df.shape)
df.head(3)
# AHD column is the target column

(303, 14)


,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes


In our dataset we have 303 patients and 13 independent variables and 1 binary target variable.

When we are working with classification problems it is always good practice to check the class balance.

In [16]:
df['AHD'].value_counts(normalize=True)

No     0.541254
Yes    0.458746
Name: AHD, dtype: float64

We see that approximately 54% of the patients are in the class 0 which refers to 'no presence' of a heart disease. Consequently, 46% of the patients have a heart disease. 

## Creating Train-Test Split

In [17]:
# For model evaluation we split our data into two parts: Train - Test

X = df.drop('AHD', axis=1)
y = df['AHD']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77, stratify=y,
                                                    test_size=.2)

In [18]:
# Let's check number of 1 and 0 in both datasets
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)
# Note that the ratios are pretty darn close

(No     0.541322
 Yes    0.458678
 Name: AHD, dtype: float64,
 No     0.540984
 Yes    0.459016
 Name: AHD, dtype: float64)

## Choosing a Perfomance Metric for Model Evaluations

__Model Selection vs Model Evaluation__

- Model Selection/Model Comparison: What is the best parameters for a given model. Between different models which one is better models the reality.

Ex: If we are working with an app that runs a machine learning algorithm model selection is choosing the process of choosing a final algorithm to deploy.


- Model Evaluation: After selecting a 'best' model with model selection how this model will perform in the 'real' case.

Ex: Model evaluation is where we want to predict how successful this algorithm will be.

[Available tools in sklearn](https://scikit-learn.org/stable/model_selection.html)

<img src= 'img/table.png' width = 450 />

## Data Prep Before Training a Model

[A good blog post on handling categorical variables](https://www.bogotobogo.com/python/scikit-learn/scikit_machine_learning_Data_Preprocessing-Missing-Data-Categorical-Data.php)

__Your Turn__

- Convert Categorical Variables to OneHotEncoding


In [19]:
steps = []

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [21]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy='most_frequent')
X_train_imputed = pd.DataFrame(si.fit_transform(X_train))
X_test_imputed = pd.DataFrame(si.transform(X_test))

In [22]:
# create an encoder object. This will help us to convert
# categorical variables to new columns
encoder = OneHotEncoder(drop='first')

# Create an columntransformer object.
# This will help us to merge transformed columns
# with the rest of the dataset.
catvars = [2,6,10,12]
ct = ColumnTransformer([('encoder', encoder, catvars)],
                       remainder='passthrough',
                       sparse_threshold=0
                      )

X_train_transformed = ct.fit_transform(X_train_imputed)

X_train_transformed

array([[0.0, 0.0, 0.0, ..., 1, 5.6, 0.0],
       [0.0, 1.0, 0.0, ..., 0, 1.4, 1.0],
       [0.0, 0.0, 0.0, ..., 0, 0.0, 0.0],
       ...,
       [0.0, 1.0, 0.0, ..., 0, 0.0, 0.0],
       [0.0, 1.0, 0.0, ..., 0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 1, 0.1, 1.0]], dtype=object)

__Don't forget!!__

- Apply the same transformations to the test data.

In [23]:
X_test_transformed = ct.transform(X_test_imputed)
X_test_transformed.shape

(61, 18)

__Scaling Features__ 

-- Let's go back to the column transformer.

[Different Scalers and Their Effect on Data](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py)

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
ss = StandardScaler()
X_train_final = ss.fit_transform(X_train_transformed)
X_test_final = ss.transform(X_test_transformed)

In [26]:
steps.append(('scaler', ss))
steps

[('scaler', StandardScaler())]

In [27]:
X_test_final.shape

(61, 18)

In [28]:
np.mean(X, axis=0)

# What do you expect if you check the means of X_test_final? Try

Age         54.438944
Sex          0.679868
RestBP     131.689769
Chol       246.693069
Fbs          0.148515
RestECG      0.990099
MaxHR      149.607261
ExAng        0.326733
Oldpeak      1.039604
Slope        1.600660
Ca           0.672241
dtype: float64

In [29]:
np.mean(X_test_final, axis=0)

array([-0.14876925, -0.05832876,  0.1100025 , -0.12964074,  0.09058001,
        0.07482549, -0.09604015, -0.03396337,  0.14632806,  0.17988506,
       -0.10971427,  0.05575147,  0.12768021, -0.17079281, -0.19117011,
        0.09119066,  0.08256576,  0.21814072])

## Model Training

[Check sklearn for documentation of Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


[For solvers](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

In [30]:
log_reg = LogisticRegression(penalty='none', max_iter=10000)
log_reg.fit(X, y_train)

ValueError: could not convert string to float: 'normal'

In [ ]:
# What is this score?
print(log_reg.score(X, y_train))

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
y_pred = log_reg.predict(X)

score = log_reg.score(X, y_train)


In [ ]:
cm = confusion_matrix(y_train, y_pred)

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5,
            square=True, cmap='Blues')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size=15)
plt.savefig('toy_Digits_ConfusionSeabornCodementor.png')
# plt.show();

## ROC Curves for Model Selection

### Plotting ROC curves

In [ ]:
log_reg_vanilla = LogisticRegression(penalty='none', max_iter=10000)

log_reg_l2 = LogisticRegression(penalty='l2', C=0.01, max_iter=10000)

In [ ]:
log_reg_vanilla.fit(X, y_train)

y_probs_vanilla = log_reg_vanilla.predict_proba(X)

In [ ]:
log_reg_l2.fit(X, y_train)
y_probs_l2 = log_reg_l2.predict_proba(X)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr_v, tpr_v, thresholds_v = roc_curve(y_train, y_probs_vanilla[:,1])
fpr_l2, tpr_l2, thresholds_l2 = roc_curve(y_train, y_probs_l2[:,1])

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')


plot_roc_curve(fpr_v, tpr_v, label='Vanilla')
plot_roc_curve(fpr_l2, tpr_l2, label='L2-Penalty')
plt.legend()
plt.show()

Also we can measure the __A__rea __U__nder __C__urve scores


In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:

roc_auc_score(y_train, y_probs_vanilla[:,1])

In [ ]:
roc_auc_score(y_train, y_probs_l2[:,1])

### The Default Measure (in most prebuilt models) - Accuracy

$$ \frac{(TP + TN)}{(TP + FP + TN + FN)} $$

Category definitions - possible outcomes in binary classification

- TP = True Positive (class 1 correctly classified as class 1) - e.g. Patient with cancer tests positive for cancer
- TN = True Negative (class 0 correctly classified as class 0) - e.g. Patient without cancer tests negative for cancer
- FP = False Positive (class 0 incorrectly classified as class 1) - e.g. Patient without cancer tests positive for cancer
- FN = False Negative (class 1 incorrectly classified as class 0) - e.g. Patient with cancer tests negative for cancer

 $$ \text{Possible misclassifications} $$

<img src='./img/type-1-type-2.jpg' width=400/>
 

Remember that Logistic Regression gives probability predictions for each class, in addition to the final classification. By default, threshold for the prediction is set to 0.5, but we can adjust that threshold.

### The AUC / ROC curve (Area Under Curve of the Receiver Operating Characteristic)

<img src='img/pop-curve.png' width=500/>


### Using Cross Validation Scores for Model Evaluation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
log_reg = LogisticRegression(penalty='none', max_iter=10000)

In [ ]:
y_scores = cross_val_score(log_reg, X, y_train, cv=3, scoring='roc_auc')

In [ ]:
y_scores